# ML Surrogates for Chemical Processes with OMLT
This notebook illustrates the use of TensorFlow Keras and OMLT to produce an ML surrogate based on data from a chemical process flowsheet.

There are several reasons to build surrogate models for complex processes, even when higher fidelity models already exist (e.g., reduce model size, improve convergence reliability, replace models with externally compiled code and make them fully-equation oriented).

In this example, we have an existing model for an auto-thermal reformer flowsheet that has been built using the IDAES-PSE package. IDAES-PSE is a Python package that is built on Pyomo and provides a framework for equation-oriented modeling and analysis of advanced energy systems. We use this package to generate data for our systems, and then we utilize this data in an optimization problem with OMLT. To learn more about IDAES, see the [IDAES-PSE Github Page](https://github.com/IDAES/IDAES-PSE) or [IDAES Read-the-docs](https://idaes-pse.readthedocs.io/en/stable/).

## The Auto-thermal Reformer Process

The figure below shows the reformer process as modeled in IDAES.

![Reformer Flowsheet](../images/reformer.png)

This model has 12 outputs of interest, the steam flowrate, the reformer duty, and the properties of the outlet stream, including temperature, pressure, and composition. We are interested modeling how these outputs change as a function of two operating (or input) variables: 
- the fraction of natural gas that bypasses the reformer
- steam to natural gas flow ratio

We have already used IDAES to generate a CSV file that contains the input and output data for 2800 data points for our system.

In this example, we will train a neural network model with sigmoid activations from our process data and then demonstrate that we can solve an optimization problem with that surrogate model. In realistic applications, this surrogate model would form part of a design or operations problem with a much larger flowsheet. 

In [26]:
%pip install omlt
%pip install onnx
%pip install onyx
%pip install ipopt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
  Using cached onyx-0.8.11.tar.gz (99 kB)
  Using cached onyx-0.8.10.tar.gz (99 kB)
  Using cached onyx-0.8.7.zip (177 kB)
  Using cached onyx-0.8.5.zip (176 kB)
  Using cached onyx-0.7.13.zip (93 kB)
  Using cached onyx-0.7.12.zip (93 kB)
  Using cached onyx-0.7.11.zip (93 kB)
  Using cached onyx-0.7.10.tar.gz (54 kB)
  Using cached onyx-0.7.8.zip (91 kB)
  Using cached onyx-0.7.7.zip (91 kB)
  Using cached onyx-0.7.6.zip (91 kB)
  Using cached onyx-0.7.5.zip (90 kB)
  Using cached onyx-0.7.4.tar.gz (52 kB)
  Using cached onyx-0.7.3.zip (90 kB)
  Using cached onyx-0.6.4.zip (89 kB)
  Using cached onyx-0.6.3.zip (89 kB)
  Using cached onyx-0.6.2.zip (89 kB)
  Using cached onyx-0.6.1.tar.gz (53 kB)
  Using cached onyx-0.5.zip (88 kB)
  Using cached onyx-0.4.7.tar.gz (52 kB)
  Using cached onyx-0.4.6.zip (88 kB)
  Using cached onyx-0.4.5.zip (88 kB)
  Using cached onyx-0.4.4.zip (88 kB)
  Using cached onyx-0.4.

ERROR: Could not find a version that satisfies the requirement onyx (from versions: 0.0.5, 0.0.17, 0.0.19, 0.0.20, 0.0.21, 0.1, 0.1.1, 0.1.3, 0.1.4, 0.1.5, 0.2, 0.2.1, 0.3, 0.3.2, 0.3.3, 0.3.4, 0.3.5, 0.3.6, 0.3.7, 0.3.8, 0.3.9, 0.3.10, 0.3.11, 0.3.12, 0.4, 0.4.1, 0.4.2, 0.4.4, 0.4.5, 0.4.6, 0.4.7, 0.5, 0.6.1, 0.6.2, 0.6.3, 0.6.4, 0.7.3, 0.7.4, 0.7.5, 0.7.6, 0.7.7, 0.7.8, 0.7.10, 0.7.11, 0.7.12, 0.7.13, 0.8.5, 0.8.7, 0.8.10, 0.8.11)
ERROR: No matching distribution found for onyx
You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
  Running setup.py clean for ipopt
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=e65549e9b07d714612754a700627ebb1fa30c1274bacbabf6bf40dfc4ddc1689
  Stored in directory: c:\users\jwiggins\appdata\local\pip\cache\wheels\2f\a0\d3\4030d9f80e6b3be787f19fc911b8e7aa462986a40ab1e4bb94
Successfully built future
Failed to build ipopt
    Running setup.py install for ipopt: started
    Running setup.py install for ipopt: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


    ERROR: Command errored out with exit status 1:
     command: 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\jwiggins\\AppData\\Local\\Temp\\pip-install-ltd0t_2y\\ipopt_74a2abb4e4b84e31bfe959785c472241\\setup.py'"'"'; __file__='"'"'C:\\Users\\jwiggins\\AppData\\Local\\Temp\\pip-install-ltd0t_2y\\ipopt_74a2abb4e4b84e31bfe959785c472241\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\jwiggins\AppData\Local\Temp\pip-pip-egg-info-0vizgmg_'
         cwd: C:\Users\jwiggins\AppData\Local\Temp\pip-install-ltd0t_2y\ipopt_74a2abb4e4b84e31bfe959785c472241\
    Complete output (13 lines):
    Traceback (most recent call last):
      File 

In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # suppress CUDA warnings from tensorflow

# import the necessary packages
from omlt import OmltBlock, OffsetScaling
from omlt.io.keras_reader import load_keras_sequential
from omlt.neuralnet import FullSpaceSmoothNNFormulation
import pyomo.environ as pyo
import pandas as pd
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [6]:
# read in our csv data
columns = ['Bypass Fraction', 'NG Steam Ratio', 'Steam Flow',
           'Reformer Duty','AR', 'C2H6', 'C3H8', 'C4H10',
           'CH4', 'CO', 'CO2', 'H2', 'H2O', 'N2']
df = pd.read_csv(r'C:\Users\jwiggins\Documents\Python_virtual_env\Supply Chain Simulation\supply_chain_simulation\supply_chain_simulation\nn_optimization\reformer.csv', usecols=columns)
print(df)

      Bypass Fraction  NG Steam Ratio  Steam Flow  Reformer Duty        AR  \
0                 0.8        0.800000    0.193898    9806.732716  0.002662   
1                 0.8        0.810526    0.196449    9846.047501  0.002660   
2                 0.8        0.821053    0.199000    9885.419259  0.002657   
3                 0.8        0.831579    0.201552    9924.849127  0.002654   
4                 0.8        0.842105    0.204103    9964.338177  0.002651   
...               ...             ...         ...            ...       ...   
2795              0.1        1.157895    1.262887   39771.876388  0.004086   
2796              0.1        1.168421    1.274368   39989.582852  0.004080   
2797              0.1        1.178947    1.285849   40207.531167  0.004073   
2798              0.1        1.189474    1.297330   40425.721366  0.004067   
2799              0.1        1.200000    1.308811   40644.153425  0.004060   

          C2H6      C3H8     C4H10       CH4        CO       CO

In [7]:
# separate the data into inputs and outputs
inputs = ['Bypass Fraction', 'NG Steam Ratio']
outputs = [ 'Steam Flow', 'Reformer Duty','AR', 'C2H6', 'C3H8', 'C4H10',
           'CH4', 'CO', 'CO2', 'H2', 'H2O', 'N2']
dfin = df[inputs]
dfout = df[outputs]

In [8]:
# We scale the data for improved training, however, we want to formulate
# our optimizaton problem on the original variables. Therefore, we keep
# the scaling parameters to use later in our optimization formulation

x_offset, x_factor = dfin.mean().to_dict(), dfin.std().to_dict()
y_offset, y_factor = dfout.mean().to_dict(), dfout.std().to_dict()

dfin = (dfin - dfin.mean()).divide(dfin.std())
dfout = (dfout - dfout.mean()).divide(dfout.std())

# capture the minimum and maximum values of the scaled inputs
# so we don't use the model outside the valid range
scaled_lb = dfin.min()[inputs].values
scaled_ub = dfin.max()[inputs].values

In [9]:
# create our Keras Sequential model
nn = Sequential(name='reformer_sigmoid_4_20')
nn.add(Dense(units=20, input_dim=len(inputs), activation='sigmoid'))
nn.add(Dense(units=20, activation='sigmoid'))
nn.add(Dense(units=20, activation='sigmoid'))
nn.add(Dense(units=20, activation='sigmoid'))
nn.add(Dense(units=len(outputs)))
nn.compile(optimizer=Adam(), loss='mse')

In [10]:
# train our model
x = dfin.values
y = dfout.values

history = nn.fit(x, y, epochs=100)

Epoch 1/100
88/88 [==============================] - 1s 1ms/step - loss: 1.0671
Epoch 2/100
88/88 [==============================] - 0s 1ms/step - loss: 0.9969
Epoch 3/100
88/88 [==============================] - 0s 1ms/step - loss: 0.9701
Epoch 4/100
88/88 [==============================] - 0s 1ms/step - loss: 0.7652
Epoch 5/100
88/88 [==============================] - 0s 1ms/step - loss: 0.3158
Epoch 6/100
88/88 [==============================] - 0s 1ms/step - loss: 0.1692
Epoch 7/100
88/88 [==============================] - 0s 2ms/step - loss: 0.1363
Epoch 8/100
88/88 [==============================] - 0s 1ms/step - loss: 0.1171
Epoch 9/100
88/88 [==============================] - 0s 1ms/step - loss: 0.1042
Epoch 10/100
88/88 [==============================] - 0s 1ms/step - loss: 0.0949
Epoch 11/100
88/88 [==============================] - 0s 1ms/step - loss: 0.0881
Epoch 12/100
88/88 [==============================] - 0s 1ms/step - loss: 0.0829
Epoch 13/100
88/88 [=================

In [11]:
# save the model to disk
# While not technically necessary, this shows how we can load a previously saved model into
# our optimization formulation)
nn.save('reformer_nn')

INFO:tensorflow:Assets written to: reformer_nn\assets


## Optimization Problem
In this small example, we will formulate a simple optimization problem that seeks to maximize the concentration of Hydrogen in the outlet while placing an upper bound on the Nitrogen concentration.

In [12]:
# first, create the Pyomo model
m = pyo.ConcreteModel()

In [18]:
# create the OmltBlock to hold the neural network model
m.reformer = OmltBlock()

    'pyomo.core.base.block._ScalarOmltBlock'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.block._ScalarOmltBlock'>). This is
    usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


In [19]:
# load the Keras model
nn_reformer = keras.models.load_model('reformer_nn', compile=False)

# Note: The neural network is in the scaled space. We want access to the
# variables in the unscaled space. Therefore, we need to tell OMLT about the
# scaling factors
scaler = OffsetScaling(
        offset_inputs={i: x_offset[inputs[i]] for i in range(len(inputs))},
        factor_inputs={i: x_factor[inputs[i]] for i in range(len(inputs))},
        offset_outputs={i: y_offset[outputs[i]] for i in range(len(outputs))},
        factor_outputs={i: y_factor[outputs[i]] for i in range(len(outputs))}
    )

scaled_input_bounds = {i: (scaled_lb[i], scaled_ub[i]) for i in range(len(inputs))}

# create a network definition from the Keras model
net = load_keras_sequential(nn_reformer, scaling_object=scaler, scaled_input_bounds=scaled_input_bounds)

# create the variables and constraints for the neural network in Pyomo
m.reformer.build_formulation(FullSpaceSmoothNNFormulation(net))

In [20]:
# now add the objective and the constraints
h2_idx = outputs.index('H2')
n2_idx = outputs.index('N2')
m.obj = pyo.Objective(expr=m.reformer.outputs[h2_idx], sense=pyo.maximize)
m.con = pyo.Constraint(expr=m.reformer.outputs[n2_idx] <= 0.34)

    'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


In [35]:
# now solve the optimization problem
solver = pyo.SolverFactory('gurobi')
status = solver.solve(m, tee=True)

    solver gurobi


ApplicationError: No executable found for solver 'gurobi'

In [15]:
print('Bypass Fraction:', pyo.value(m.reformer.inputs[0]))
print('NG Steam Ratio:', pyo.value(m.reformer.inputs[1]))
print('H2 Concentration:', pyo.value(m.reformer.outputs[h2_idx]))
print('N2 Concentration:', pyo.value(m.reformer.outputs[n2_idx]))

Bypass Fraction: 0.10000025307452928
NG Steam Ratio: 1.1197517732543654
H2 Concentration: 0.3313070992873072
N2 Concentration: 0.34000000393182694


In [33]:
idaes get-extensions

SyntaxError: invalid syntax (4269050250.py, line 1)

In [34]:
%pyomo help --solvers


UsageError: Line magic function `%pyomo` not found.
